In [3]:
# Aviation Hybrid Cryptography Dashboard - Enhanced with Download Features

# ╔═══════════════════════════════════════════════════════════════════╗
# ║  Aviation Hybrid Cryptography Dashboard - Enhanced                 ║
# ║  ECDH + Kyber-768 • ECDSA + Falcon-512 • AES-256-GCM            ║
# ║  Quantum-Secure Communication for ATC ↔ Aircraft                  ║
# ║  With Enhanced Download Capabilities                              ║
# ╚═══════════════════════════════════════════════════════════════════╝

"""
This notebook implements a quantum-secure communication system for aviation,
combining classical and post-quantum cryptography to protect against both
current and future quantum threats.

ENHANCED: Added comprehensive download functionality for all outputs,
especially detailed demo reports and cryptographic artifacts.

SECTIONS:
1. Dependencies and Imports
2. Utility Functions (Enhanced with download features)
3. Cryptographic Core Classes
4. UI Helper Functions
5. Screen Functions
   5.1 Home Screen
   5.2 Key Generation Screen
   5.3 Key Exchange Screen
   5.4 Signature Screen
   5.5 Messaging Screen
   5.6 Demo Screen (Enhanced with comprehensive reporting)
6. Global State Management
7. Main Execution
"""

# ============================================================================
# SECTION 1: DEPENDENCIES AND IMPORTS
# ============================================================================

# First, let's install all required packages if they're not already installed
# This ensures the notebook works in any environment (Google Colab, local, etc.)

import importlib
import subprocess
import sys

# List of required packages for our hybrid cryptography system
required_packages = [
    "oqs",           # Open Quantum Safe library for post-quantum algorithms
    "cryptography",  # Python cryptography library for classical algorithms
    "pycryptodome",  # Additional cryptographic primitives
    "ipywidgets"     # Interactive widgets for Jupyter notebooks
]

# Install missing packages
print("Checking and installing dependencies...")
for package in required_packages:
    if importlib.util.find_spec(package) is None:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✓ {package} installed")
    else:
        print(f"✓ {package} already available")

# Now import all required modules
import oqs  # Post-quantum cryptography algorithms (Kyber, Falcon)
import ipywidgets as w  # Interactive UI widgets
from IPython.display import display, clear_output, HTML
import json
import time
import base64
import hashlib
import os
from datetime import datetime

# Classical cryptography imports
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend

# Symmetric encryption imports
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

print("\n✅ All dependencies loaded successfully!")

# ============================================================================
# SECTION 2: UTILITY FUNCTIONS - ENHANCED WITH DOWNLOAD FEATURES
# ============================================================================

def encode_b64(data: bytes) -> str:
    """
    Convert binary data to Base64 string for safe transport/display.
    
    Args:
        data: Binary data (keys, ciphertext, etc.)
        
    Returns:
        Base64 encoded string
    """
    return base64.b64encode(data).decode('utf-8')

def decode_b64(data: str) -> bytes:
    """
    Convert Base64 string back to binary data.
    
    Args:
        data: Base64 encoded string
        
    Returns:
        Original binary data
    """
    return base64.b64decode(data.encode('utf-8'))

def create_download_link(data: bytes, filename: str, button_text: str = None) -> w.HTML:
    """
    Create a downloadable link for binary data in Jupyter notebook.
    
    This function creates an HTML anchor tag with embedded Base64 data
    that allows users to download keys, signatures, etc.
    
    Args:
        data: Binary data to make downloadable
        filename: Suggested filename for download
        button_text: Optional custom button text
        
    Returns:
        HTML widget with download link
    """
    data_url = f"data:application/octet-stream;base64,{encode_b64(data)}"
    
    # Styling for a nice-looking download button
    style = """
        background: #28a745;
        color: white;
        padding: 8px 16px;
        border-radius: 5px;
        text-decoration: none;
        display: inline-block;
        margin: 5px;
        font-weight: bold;
        transition: background 0.3s;
    """
    
    display_text = button_text or f"📥 {filename}"
    html = f'<a href="{data_url}" download="{filename}" style="{style}" onmouseover="this.style.background=\'#218838\'" onmouseout="this.style.background=\'#28a745\'">{display_text}</a>'
    
    return w.HTML(html)

def create_report_download(content: str, filename: str, format_type: str = "txt") -> w.HTML:
    """
    Create a downloadable report in various formats.
    
    Args:
        content: Report content as string
        filename: Base filename (without extension)
        format_type: 'txt', 'json', or 'html'
        
    Returns:
        HTML widget with download link
    """
    if format_type == "json":
        # Ensure content is valid JSON
        try:
            json_content = json.loads(content) if isinstance(content, str) else content
            data = json.dumps(json_content, indent=2).encode('utf-8')
            filename = f"{filename}.json"
        except:
            data = content.encode('utf-8')
            filename = f"{filename}.json"
    elif format_type == "html":
        # Wrap content in basic HTML structure
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>{filename}</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; }}
                h1, h2, h3 {{ color: #2c3e50; }}
                .success {{ color: #27ae60; }}
                .error {{ color: #e74c3c; }}
                .info {{ background: #ecf0f1; padding: 10px; border-radius: 5px; }}
                pre {{ background: #f8f9fa; padding: 10px; border-radius: 5px; overflow-x: auto; }}
            </style>
        </head>
        <body>
            {content}
        </body>
        </html>
        """
        data = html_content.encode('utf-8')
        filename = f"{filename}.html"
    else:
        # Default to text
        data = content.encode('utf-8')
        filename = f"{filename}.txt"
    
    return create_download_link(data, filename)

def generate_demo_report(demo_data: dict) -> str:
    """
    Generate a comprehensive demo report in HTML format.
    
    Args:
        demo_data: Dictionary containing demo execution data
        
    Returns:
        HTML formatted report string
    """
    report = f"""
    <h1>Aviation Hybrid Cryptography Demo Report</h1>
    <p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
    
    <h2>Executive Summary</h2>
    <div class="info">
        <p><strong>Scenario:</strong> {demo_data.get('scenario', 'N/A')}</p>
        <p><strong>Status:</strong> <span class="success">{demo_data.get('status', 'Completed Successfully')}</span></p>
        <p><strong>Total Messages:</strong> {len(demo_data.get('messages', []))}</p>
        <p><strong>Entities:</strong> {demo_data.get('atc_name', 'ATC')} ↔ {demo_data.get('aircraft_name', 'Aircraft')}</p>
    </div>
    
    <h2>Cryptographic Configuration</h2>
    <ul>
        <li><strong>Key Exchange:</strong> ECDH (P-384) + Kyber-768</li>
        <li><strong>Digital Signatures:</strong> ECDSA (P-384) + Falcon-512</li>
        <li><strong>Encryption:</strong> AES-256-GCM</li>
        <li><strong>Hash Functions:</strong> SHA-256/512 with HKDF</li>
    </ul>
    
    <h2>Communication Log</h2>
    """
    
    # Add messages
    for i, msg in enumerate(demo_data.get('messages', []), 1):
        report += f"""
        <div style="margin: 20px 0; padding: 15px; background: #f8f9fa; border-left: 3px solid #3498db;">
            <h3>Message {i}</h3>
            <p><strong>From:</strong> {msg.get('sender', 'Unknown')}</p>
            <p><strong>To:</strong> {msg.get('receiver', 'Unknown')}</p>
            <p><strong>Content:</strong> {msg.get('message', 'N/A')}</p>
            <p><strong>Timestamp:</strong> {msg.get('timestamp', 'N/A')}</p>
            <p><strong>Security:</strong></p>
            <ul>
                <li>Encrypted: ✅</li>
                <li>Signed: ✅</li>
                <li>Verified: {msg.get('verified', '✅')}</li>
            </ul>
        </div>
        """
    
    # Add performance metrics if available
    if 'performance' in demo_data:
        report += """
        <h2>Performance Metrics</h2>
        <table border="1" style="border-collapse: collapse; width: 100%;">
            <tr><th>Operation</th><th>Time (seconds)</th></tr>
        """
        for op, time_val in demo_data['performance'].items():
            report += f"<tr><td>{op}</td><td>{time_val:.6f}</td></tr>"
        report += "</table>"
    
    report += """
    <h2>Security Analysis</h2>
    <div class="info">
        <p>This communication session provides protection against:</p>
        <ul>
            <li><strong>Classical Attacks:</strong> Protected by ECDH/ECDSA with 384-bit curves</li>
            <li><strong>Quantum Attacks:</strong> Protected by Kyber-768 and Falcon-512</li>
            <li><strong>Replay Attacks:</strong> Each message includes unique nonces and timestamps</li>
            <li><strong>Tampering:</strong> GCM mode provides authenticated encryption</li>
        </ul>
    </div>
    """
    
    return report

def display_timing_metrics(timings: dict) -> None:
    """
    Display performance metrics in a formatted table.
    
    Args:
        timings: Dictionary of operation names to execution times
    """
    # Create HTML table for timing display
    html = """
    <table style='border-collapse: collapse; width: 100%; margin: 10px 0;'>
        <tr style='background: #f8f9fa; border-bottom: 2px solid #dee2e6;'>
            <th style='padding: 10px; text-align: left;'>Operation</th>
            <th style='padding: 10px; text-align: right;'>Time (seconds)</th>
        </tr>
    """
    
    # Add each timing entry
    for operation, duration in timings.items():
        # Color code based on duration
        color = "#28a745" if duration < 0.01 else "#ffc107" if duration < 0.1 else "#dc3545"
        html += f"""
        <tr style='border-bottom: 1px solid #dee2e6;'>
            <td style='padding: 8px;'>{operation}</td>
            <td style='padding: 8px; text-align: right; color: {color}; font-weight: bold;'>
                {duration:.6f}s
            </td>
        </tr>
        """
    
    # Add total time
    total = sum(timings.values())
    html += f"""
        <tr style='background: #f8f9fa; font-weight: bold;'>
            <td style='padding: 8px;'>Total Time</td>
            <td style='padding: 8px; text-align: right;'>{total:.6f}s</td>
        </tr>
    </table>
    """
    
    display(w.HTML(html))

def create_session_archive(atc_entity, aircraft_entity, messages: list = None) -> bytes:
    """
    Create a comprehensive archive of the entire session.
    
    Args:
        atc_entity: ATC entity object
        aircraft_entity: Aircraft entity object
        messages: List of exchanged messages
        
    Returns:
        ZIP file as bytes containing all session data
    """
    import zipfile
    import io
    
    # Create in-memory ZIP file
    zip_buffer = io.BytesIO()
    
    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        # Add timestamp file
        timestamp_info = f"Session Archive Created: {datetime.now().isoformat()}\n"
        zip_file.writestr("session_info.txt", timestamp_info)
        
        # Add public keys
        if atc_entity:
            atc_keys = json.dumps(atc_entity.export_public_keys(), indent=2)
            zip_file.writestr("atc_public_keys.json", atc_keys)
        
        if aircraft_entity:
            aircraft_keys = json.dumps(aircraft_entity.export_public_keys(), indent=2)
            zip_file.writestr("aircraft_public_keys.json", aircraft_keys)
        
        # Add message log
        if messages:
            message_log = json.dumps(messages, indent=2)
            zip_file.writestr("message_log.json", message_log)
        
        # Add configuration info
        config = {
            "algorithms": {
                "key_exchange": ["ECDH-P384", "Kyber-768"],
                "signatures": ["ECDSA-P384", "Falcon-512"],
                "encryption": "AES-256-GCM",
                "hash": ["SHA-256", "SHA-512", "HKDF"]
            },
            "security_level": {
                "classical": "192-bit",
                "post_quantum": "NIST Level 3"
            }
        }
        zip_file.writestr("crypto_config.json", json.dumps(config, indent=2))
    
    zip_buffer.seek(0)
    return zip_buffer.read()

# ============================================================================
# SECTION 3: CRYPTOGRAPHIC CORE CLASSES
# ============================================================================

class AviationEntity:
    """
    Represents an aviation entity (ATC or Aircraft) with hybrid cryptographic capabilities.
    
    This class encapsulates all cryptographic operations needed for secure
    aviation communication, including key generation, exchange, signing, and encryption.
    """
    
    def __init__(self, entity_name: str, entity_type: str = "ATC"):
        """
        Initialize an aviation entity with hybrid cryptographic capabilities.
        
        Args:
            entity_name: Unique identifier (e.g., "JFK_Tower", "UA123")
            entity_type: Either "ATC" or "Aircraft"
        """
        self.entity_name = entity_name
        self.entity_type = entity_type
        self.keys = {
            "classical": {},
            "post_quantum": {}
        }
        self.session_key = None
        self.peer_public_keys = None
        
        # Timing metrics for performance analysis
        self.last_operation_timings = {}
        
        # Message history for audit trail
        self.message_history = []
        
        print(f"🛩️ Initializing {entity_type}: {entity_name}")
    
    def generate_hybrid_keys(self):
        """
        Generate both classical and post-quantum key pairs.
        
        This method creates:
        1. ECDH key pair for classical key exchange
        2. ECDSA key pair for classical signatures
        3. Kyber-768 key pair for post-quantum key exchange
        4. Falcon-512 key pair for post-quantum signatures
        """
        timings = {}
        
        print(f"\n🔐 Generating hybrid keys for {self.entity_name}...")
        
        # ===== Classical Key Generation =====
        
        # Generate ECDH key pair (for key exchange)
        start_time = time.time()
        self.keys["classical"]["ecdh_private"] = ec.generate_private_key(
            ec.SECP384R1(),  # NIST P-384 curve, provides ~192-bit security
            default_backend()
        )
        self.keys["classical"]["ecdh_public"] = self.keys["classical"]["ecdh_private"].public_key()
        timings["ECDH Key Generation"] = time.time() - start_time
        print("  ✓ ECDH keys generated (NIST P-384)")
        
        # Generate ECDSA key pair (for signatures)
        start_time = time.time()
        self.keys["classical"]["ecdsa_private"] = ec.generate_private_key(
            ec.SECP384R1(),  # Same curve for consistency
            default_backend()
        )
        self.keys["classical"]["ecdsa_public"] = self.keys["classical"]["ecdsa_private"].public_key()
        timings["ECDSA Key Generation"] = time.time() - start_time
        print("  ✓ ECDSA keys generated (NIST P-384)")
        
        # ===== Post-Quantum Key Generation =====
        
        # Generate Kyber-768 key pair (for key exchange)
        # Kyber-768 provides ~192-bit post-quantum security
        start_time = time.time()
        self.keys["post_quantum"]["kyber"] = oqs.KeyEncapsulation("Kyber768")
        self.keys["post_quantum"]["kyber_public"] = self.keys["post_quantum"]["kyber"].generate_keypair()
        self.keys["post_quantum"]["kyber_secret"] = self.keys["post_quantum"]["kyber"].export_secret_key()
        timings["Kyber-768 Key Generation"] = time.time() - start_time
        print("  ✓ Kyber-768 keys generated")
        
        # Generate Falcon-512 key pair (for signatures)
        # Falcon-512 provides NIST Level 1 security
        start_time = time.time()
        self.keys["post_quantum"]["falcon"] = oqs.Signature("Falcon-512")
        self.keys["post_quantum"]["falcon_public"] = self.keys["post_quantum"]["falcon"].generate_keypair()
        self.keys["post_quantum"]["falcon_secret"] = self.keys["post_quantum"]["falcon"].export_secret_key()
        timings["Falcon-512 Key Generation"] = time.time() - start_time
        print("  ✓ Falcon-512 keys generated")
        
        self.last_operation_timings = timings
        print(f"\n✅ All keys generated successfully for {self.entity_name}")
        
        return timings
    
    def export_public_keys(self) -> dict:
        """
        Export all public keys in a format suitable for sharing.
        
        Returns:
            Dictionary containing all public keys in transportable format
        """
        # Serialize classical keys to PEM format
        ecdh_public_pem = self.keys["classical"]["ecdh_public"].public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ).decode('utf-8')
        
        ecdsa_public_pem = self.keys["classical"]["ecdsa_public"].public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ).decode('utf-8')
        
        # Post-quantum keys are already in bytes, encode to base64 for transport
        return {
            "entity_name": self.entity_name,
            "entity_type": self.entity_type,
            "timestamp": datetime.now().isoformat(),
            "classical": {
                "ecdh_public": ecdh_public_pem,
                "ecdsa_public": ecdsa_public_pem
            },
            "post_quantum": {
                "kyber_public": encode_b64(self.keys["post_quantum"]["kyber_public"]),
                "falcon_public": encode_b64(self.keys["post_quantum"]["falcon_public"])
            }
        }
    
    def import_peer_public_keys(self, peer_keys: dict):
        """
        Import and store peer's public keys.
        
        Args:
            peer_keys: Dictionary containing peer's public keys
        """
        self.peer_public_keys = peer_keys
        print(f"✓ Imported public keys from {peer_keys['entity_name']}")
    
    def perform_hybrid_key_exchange(self, initiator: bool = True) -> bytes:
        """
        Perform hybrid key exchange using ECDH + Kyber.
        
        This method combines classical ECDH and post-quantum Kyber to create
        a quantum-secure shared secret. Even if quantum computers break ECDH,
        Kyber will maintain security.
        
        Args:
            initiator: True if this entity initiates the exchange
            
        Returns:
            The derived hybrid session key
        """
        if not self.peer_public_keys:
            raise ValueError("No peer public keys available. Import peer keys first!")
        
        timings = {}
        print(f"\n🔄 Performing hybrid key exchange...")
        
        # ===== Classical Key Exchange (ECDH) =====
        start_time = time.time()
        
        # Deserialize peer's ECDH public key
        peer_ecdh_public = serialization.load_pem_public_key(
            self.peer_public_keys["classical"]["ecdh_public"].encode('utf-8'),
            backend=default_backend()
        )
        
        # Perform ECDH to get classical shared secret
        classical_shared_secret = self.keys["classical"]["ecdh_private"].exchange(
            ec.ECDH(),
            peer_ecdh_public
        )
        
        timings["ECDH Key Exchange"] = time.time() - start_time
        print(f"  ✓ Classical shared secret established (ECDH)")
        print(f"    Length: {len(classical_shared_secret)} bytes")
        
        # ===== Post-Quantum Key Exchange (Kyber) =====
        start_time = time.time()
        
        if initiator:
            # Initiator: Encapsulate a secret using peer's Kyber public key
            peer_kyber_public = decode_b64(self.peer_public_keys["post_quantum"]["kyber_public"])
            
            # Generate ciphertext and shared secret
            self.kyber_ciphertext, pq_shared_secret = self.keys["post_quantum"]["kyber"].encap_secret(
                peer_kyber_public
            )
            
            print(f"  ✓ Kyber ciphertext generated (initiator)")
            print(f"    Ciphertext length: {len(self.kyber_ciphertext)} bytes")
            
        else:
            # Responder: Decapsulate to recover the shared secret
            if not hasattr(self, 'kyber_ciphertext'):
                raise ValueError("No Kyber ciphertext received!")
            
            pq_shared_secret = self.keys["post_quantum"]["kyber"].decap_secret(
                self.kyber_ciphertext
            )
            
            print(f"  ✓ Kyber shared secret recovered (responder)")
        
        timings["Kyber Key Exchange"] = time.time() - start_time
        print(f"    PQ Secret length: {len(pq_shared_secret)} bytes")
        
        # ===== Hybrid Key Derivation =====
        start_time = time.time()
        
        # Combine both secrets: classical || post-quantum
        combined_secret = classical_shared_secret + pq_shared_secret
        
        # Use HKDF (HMAC-based Key Derivation Function) to derive final key
        # This ensures that the final key has full entropy from both sources
        hkdf = HKDF(
            algorithm=hashes.SHA256(),
            length=32,  # 256-bit key for AES-256
            salt=None,  # Could add salt for additional security
            info=b'aviation-hybrid-session-key',  # Context-specific info
            backend=default_backend()
        )
        
        self.session_key = hkdf.derive(combined_secret)
        
        timings["Key Derivation (HKDF)"] = time.time() - start_time
        
        print(f"\n✅ Hybrid session key established!")
        print(f"   Final key length: {len(self.session_key) * 8} bits")
        
        self.last_operation_timings = timings
        return self.session_key
    
    def sign_message(self, message: str) -> dict:
        """
        Create hybrid digital signature using combined ECDSA + Falcon approach.
        
        This implements a true hybrid signature by:
        1. Creating a combined hash of the message using both classical and PQ methods
        2. Signing the combined hash with both algorithms
        3. Combining the results into a single hybrid signature
        
        Args:
            message: The message to sign
            
        Returns:
            Dictionary containing hybrid signature and metadata
        """
        timings = {}
        message_bytes = message.encode('utf-8')
        
        print(f"\n✍️ Creating hybrid signature for message...")
        print(f"   Message: '{message[:50]}{'...' if len(message) > 50 else ''}'")
        
        # ===== Step 1: Create Combined Hash =====
        start_time = time.time()
        
        # Create two different hashes
        sha256_hash = hashlib.sha256(message_bytes).digest()
        sha512_hash = hashlib.sha512(message_bytes).digest()
        
        # Combine the hashes
        combined_hash = sha256_hash + sha512_hash
        
        # Use HKDF to create a final hash
        hkdf = HKDF(
            algorithm=hashes.SHA256(),
            length=32,  # 256-bit output
            salt=b'hybrid-signature-salt',
            info=b'aviation-hybrid-hash',
            backend=default_backend()
        )
        
        hybrid_hash = hkdf.derive(combined_hash)
        
        timings["Hybrid Hash Creation"] = time.time() - start_time
        print(f"  ✓ Hybrid hash created")
        
        # ===== Step 2: Sign with ECDSA =====
        start_time = time.time()
        
        # Sign the hybrid hash with ECDSA
        ecdsa_sig = self.keys["classical"]["ecdsa_private"].sign(
            hybrid_hash,
            ec.ECDSA(hashes.SHA256())
        )
        
        timings["ECDSA Signing"] = time.time() - start_time
        
        # ===== Step 3: Sign with Falcon =====
        start_time = time.time()
        
        # Sign the hybrid hash with Falcon
        falcon_sig = self.keys["post_quantum"]["falcon"].sign(hybrid_hash)
        
        timings["Falcon Signing"] = time.time() - start_time
        
        # ===== Step 4: Combine Signatures =====
        start_time = time.time()
        
        # Combine both signatures using XOR and HKDF
        # First, ensure both signatures are the same length by padding/truncating
        max_len = max(len(ecdsa_sig), len(falcon_sig))
        
        # Pad shorter signature with zeros
        ecdsa_padded = ecdsa_sig + b'\x00' * (max_len - len(ecdsa_sig))
        falcon_padded = falcon_sig + b'\x00' * (max_len - len(falcon_sig))
        
        # XOR the signatures
        xor_sig = bytes(a ^ b for a, b in zip(ecdsa_padded, falcon_padded))
        
        # Create final hybrid signature using HKDF
        hkdf_final = HKDF(
            algorithm=hashes.SHA512(),
            length=64,  # 512-bit signature
            salt=b'hybrid-signature-final',
            info=b'aviation-signature',
            backend=default_backend()
        )
        
        # Include original signature lengths for verification
        hybrid_signature = hkdf_final.derive(xor_sig)
        
        timings["Signature Combination"] = time.time() - start_time
        
        print(f"  ✓ Hybrid signature created")
        print(f"    Final signature length: {len(hybrid_signature)} bytes")
        
        self.last_operation_timings = timings
        
        # Store components needed for verification
        signed_data = {
            "message": message,
            "timestamp": datetime.now().isoformat(),
            "signer": self.entity_name,
            "hybrid_signature": encode_b64(hybrid_signature),
            "components": {
                "ecdsa": encode_b64(ecdsa_sig),
                "falcon": encode_b64(falcon_sig),
                "hybrid_hash": encode_b64(hybrid_hash)
            },
            "algorithm": "Hybrid-ECDSA-Falcon-SHA256/512"
        }
        
        # Add to message history
        self.message_history.append({
            "type": "signature",
            "data": signed_data,
            "timestamp": datetime.now().isoformat()
        })
        
        return signed_data
    
    def verify_signature(self, signed_data: dict) -> tuple:
        """
        Verify hybrid signature using peer's public keys.
        
        This verifies the hybrid signature by:
        1. Recreating the hybrid hash
        2. Verifying both component signatures
        3. Validating the combined signature
        
        Args:
            signed_data: Dictionary containing message and signatures
            
        Returns:
            Tuple of (is_valid, verification_details)
        """
        if not self.peer_public_keys:
            raise ValueError("No peer public keys available!")
        
        timings = {}
        message_bytes = signed_data["message"].encode('utf-8')
        
        print(f"\n🔍 Verifying hybrid signature from {signed_data['signer']}...")
        
        # ===== Step 1: Recreate Hybrid Hash =====
        start_time = time.time()
        
        # Create the same combined hash as during signing
        sha256_hash = hashlib.sha256(message_bytes).digest()
        sha512_hash = hashlib.sha512(message_bytes).digest()
        combined_hash = sha256_hash + sha512_hash
        
        hkdf = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b'hybrid-signature-salt',
            info=b'aviation-hybrid-hash',
            backend=default_backend()
        )
        
        hybrid_hash = hkdf.derive(combined_hash)
        
        timings["Hash Recreation"] = time.time() - start_time
        
        # Verify hash matches
        hash_matches = hybrid_hash == decode_b64(signed_data["components"]["hybrid_hash"])
        print(f"  {'✓' if hash_matches else '❌'} Hash verification: {'MATCH' if hash_matches else 'MISMATCH'}")
        
        # ===== Step 2: Verify ECDSA Component =====
        start_time = time.time()
        
        ecdsa_valid = False
        try:
            # Load peer's ECDSA public key
            peer_ecdsa_public = serialization.load_pem_public_key(
                self.peer_public_keys["classical"]["ecdsa_public"].encode('utf-8'),
                backend=default_backend()
            )
            
            # Verify ECDSA signature on hybrid hash
            peer_ecdsa_public.verify(
                decode_b64(signed_data["components"]["ecdsa"]),
                hybrid_hash,
                ec.ECDSA(hashes.SHA256())
            )
            
            ecdsa_valid = True
            print("  ✓ ECDSA component: VALID")
            
        except Exception as e:
            print(f"  ❌ ECDSA component: INVALID ({str(e)})")
        
        timings["ECDSA Verification"] = time.time() - start_time
        
        # ===== Step 3: Verify Falcon Component =====
        start_time = time.time()
        
        falcon_valid = False
        try:
            # Create Falcon verifier
            falcon_verifier = oqs.Signature("Falcon-512")
            
            # Verify Falcon signature on hybrid hash
            falcon_valid = falcon_verifier.verify(
                hybrid_hash,
                decode_b64(signed_data["components"]["falcon"]),
                decode_b64(self.peer_public_keys["post_quantum"]["falcon_public"])
            )
            
            print(f"  {'✓' if falcon_valid else '❌'} Falcon component: {'VALID' if falcon_valid else 'INVALID'}")
            
        except Exception as e:
            falcon_valid = False
            print(f"  ❌ Falcon component: INVALID ({str(e)})")
        
        timings["Falcon Verification"] = time.time() - start_time
        
        # ===== Step 4: Verify Combined Signature =====
        start_time = time.time()
        
        # All components must be valid
        overall_valid = hash_matches and ecdsa_valid and falcon_valid
        
        if overall_valid:
            # Verify the hybrid signature construction
            ecdsa_sig = decode_b64(signed_data["components"]["ecdsa"])
            falcon_sig = decode_b64(signed_data["components"]["falcon"])
            
            # Recreate the combination process
            max_len = max(len(ecdsa_sig), len(falcon_sig))
            ecdsa_padded = ecdsa_sig + b'\x00' * (max_len - len(ecdsa_sig))
            falcon_padded = falcon_sig + b'\x00' * (max_len - len(falcon_sig))
            xor_sig = bytes(a ^ b for a, b in zip(ecdsa_padded, falcon_padded))
            
            hkdf_final = HKDF(
                algorithm=hashes.SHA512(),
                length=64,
                salt=b'hybrid-signature-final',
                info=b'aviation-signature',
                backend=default_backend()
            )
            
            expected_hybrid = hkdf_final.derive(xor_sig)
            received_hybrid = decode_b64(signed_data["hybrid_signature"])
            
            hybrid_matches = expected_hybrid == received_hybrid
            overall_valid = overall_valid and hybrid_matches
            
            print(f"  {'✓' if hybrid_matches else '❌'} Hybrid signature: {'VALID' if hybrid_matches else 'INVALID'}")
        
        timings["Hybrid Verification"] = time.time() - start_time
        
        print(f"\n{'✅' if overall_valid else '❌'} Overall verification: {'PASSED' if overall_valid else 'FAILED'}")
        
        self.last_operation_timings = timings
        
        return overall_valid, {
            "hash_valid": hash_matches,
            "ecdsa_valid": ecdsa_valid,
            "falcon_valid": falcon_valid,
            "overall_valid": overall_valid,
            "algorithm": signed_data.get('algorithm', 'Unknown'),
            "timings": timings
        }
    
    def encrypt_message(self, plaintext: str) -> dict:
        """
        Encrypt message using AES-256-GCM with the hybrid session key.
        
        GCM (Galois/Counter Mode) provides both confidentiality and authenticity,
        making it ideal for secure aviation communications.
        
        Args:
            plaintext: Message to encrypt
            
        Returns:
            Dictionary containing ciphertext and encryption metadata
        """
        if not self.session_key:
            raise ValueError("No session key established! Perform key exchange first.")
        
        start_time = time.time()
        
        print(f"\n🔒 Encrypting message with AES-256-GCM...")
        print(f"   Plaintext: '{plaintext[:50]}{'...' if len(plaintext) > 50 else ''}'")
        
        # Generate random 96-bit nonce (GCM standard)
        nonce = get_random_bytes(12)
        
        # Create AES-GCM cipher
        cipher = AES.new(self.session_key, AES.MODE_GCM, nonce=nonce)
        
        # Add associated data (not encrypted but authenticated)
        # This could include headers, timestamps, etc.
        associated_data = f"{self.entity_name}:{datetime.now().isoformat()}".encode('utf-8')
        cipher.update(associated_data)
        
        # Encrypt the message
        ciphertext = cipher.encrypt(plaintext.encode('utf-8'))
        
        # Get the authentication tag
        auth_tag = cipher.digest()
        
        encryption_time = time.time() - start_time
        
        print(f"  ✓ Message encrypted")
        print(f"    Ciphertext length: {len(ciphertext)} bytes")
        print(f"    Auth tag length: {len(auth_tag)} bytes")
        print(f"    Time: {encryption_time:.6f}s")
        
        encrypted_data = {
            "ciphertext": encode_b64(ciphertext),
            "nonce": encode_b64(nonce),
            "auth_tag": encode_b64(auth_tag),
            "associated_data": associated_data.decode('utf-8'),
            "timestamp": datetime.now().isoformat(),
            "sender": self.entity_name
        }
        
        # Add to message history
        self.message_history.append({
            "type": "encrypted_message",
            "data": encrypted_data,
            "plaintext": plaintext,  # Store for demo purposes only
            "timestamp": datetime.now().isoformat()
        })
        
        return encrypted_data
    
    def decrypt_message(self, encrypted_data: dict) -> str:
        """
        Decrypt message using AES-256-GCM with the hybrid session key.
        
        Args:
            encrypted_data: Dictionary containing ciphertext and metadata
            
        Returns:
            Decrypted plaintext message
        """
        if not self.session_key:
            raise ValueError("No session key established!")
        
        start_time = time.time()
        
        print(f"\n🔓 Decrypting message from {encrypted_data['sender']}...")
        
        try:
            # Decode components
            ciphertext = decode_b64(encrypted_data["ciphertext"])
            nonce = decode_b64(encrypted_data["nonce"])
            auth_tag = decode_b64(encrypted_data["auth_tag"])
            associated_data = encrypted_data["associated_data"].encode('utf-8')
            
            # Create AES-GCM cipher for decryption
            cipher = AES.new(self.session_key, AES.MODE_GCM, nonce=nonce)
            
            # Set associated data
            cipher.update(associated_data)
            
            # Decrypt and verify
            plaintext = cipher.decrypt_and_verify(ciphertext, auth_tag)
            
            decryption_time = time.time() - start_time
            
            plaintext_str = plaintext.decode('utf-8')
            print(f"  ✓ Message decrypted successfully")
            print(f"    Plaintext: '{plaintext_str[:50]}{'...' if len(plaintext_str) > 50 else ''}'")
            print(f"    Time: {decryption_time:.6f}s")
            
            return plaintext_str
            
        except Exception as e:
            print(f"  ❌ Decryption failed: {str(e)}")
            raise
    
    def export_message_history(self) -> str:
        """
        Export complete message history as JSON.
        
        Returns:
            JSON string of message history
        """
        return json.dumps(self.message_history, indent=2)

# ============================================================================
# SECTION 4: UI HELPER FUNCTIONS
# ============================================================================

def status_box(message: str, status_type: str = "info") -> w.HTML:
    """
    Create a styled status message box.
    
    Args:
        message: Message to display
        status_type: One of 'success', 'error', 'warning', 'info'
        
    Returns:
        HTML widget with styled message
    """
    # Define color schemes for different status types
    styles = {
        "success": {
            "bg": "#d4edda",
            "border": "#c3e6cb",
            "color": "#155724",
            "icon": "✅"
        },
        "error": {
            "bg": "#f8d7da",
            "border": "#f5c6cb",
            "color": "#721c24",
            "icon": "❌"
        },
        "warning": {
            "bg": "#fff3cd",
            "border": "#ffeeba",
            "color": "#856404",
            "icon": "⚠️"
        },
        "info": {
            "bg": "#d1ecf1",
            "border": "#bee5eb",
            "color": "#0c5460",
            "icon": "ℹ️"
        }
    }
    
    style = styles.get(status_type, styles["info"])
    
    html = f"""
    <div style='
        background: {style["bg"]};
        border: 1px solid {style["border"]};
        color: {style["color"]};
        padding: 12px 20px;
        border-radius: 5px;
        margin: 10px 0;
        font-size: 14px;
    '>
        <strong>{style["icon"]} {message}</strong>
    </div>
    """
    
    return w.HTML(html)

def back_button():
    """
    Create a 'Back to Home' button for navigation.
    """
    btn = w.Button(
        description="← Back to Home",
        button_style="info",
        icon="home",
        layout=w.Layout(width="150px")
    )
    btn.on_click(lambda b: home_screen())
    display(btn)

# ============================================================================
# SECTION 5: SCREEN FUNCTIONS
# ============================================================================

# ============================================================================
# SECTION 5.1: HOME SCREEN
# ============================================================================

def home_screen():
    """
    Display the main home screen with navigation options.
    """
    clear_output(wait=True)
    
    # Header with aviation theme
    header_html = """
    <div style='text-align: center; margin: 20px 0;'>
        <h1 style='color: #2c3e50; margin-bottom: 10px;'>
            ✈️ Aviation Hybrid Cryptography System
        </h1>
        <p style='color: #7f8c8d; font-size: 18px;'>
            Quantum-Secure Communication for Air Traffic Control
        </p>
        <div style='margin: 20px 0; padding: 20px; background: #ecf0f1; border-radius: 10px;'>
            <p style='margin: 5px 0;'><strong>ECDH + Kyber-768</strong> → Hybrid Key Exchange</p>
            <p style='margin: 5px 0;'><strong>ECDSA + Falcon-512</strong> → Hybrid Digital Signatures</p>
            <p style='margin: 5px 0;'><strong>AES-256-GCM</strong> → Authenticated Encryption</p>
        </div>
    </div>
    """
    
    display(w.HTML(header_html))
    
    # Status display
    if current_atc and current_aircraft:
        status_html = f"""
        <div style='background: #d4edda; border: 1px solid #c3e6cb; padding: 15px; 
                    border-radius: 5px; margin: 20px 0;'>
            <h3 style='color: #155724; margin: 0 0 10px 0;'>✅ System Status: Ready</h3>
            <div style='display: flex; justify-content: space-around;'>
                <div>
                    <strong>🏢 ATC:</strong> {current_atc.entity_name}<br>
                    Keys: Generated ✓
                </div>
                <div>
                    <strong>✈️ Aircraft:</strong> {current_aircraft.entity_name}<br>
                    Keys: Generated ✓
                </div>
            </div>
        </div>
        """
    else:
        status_html = """
        <div style='background: #f8d7da; border: 1px solid #f5c6cb; padding: 15px; 
                    border-radius: 5px; margin: 20px 0;'>
            <h3 style='color: #721c24; margin: 0;'>⚠️ System Status: Not Initialized</h3>
            <p style='margin: 10px 0 0 0;'>Please generate keys for both ATC and Aircraft</p>
        </div>
        """
    
    display(w.HTML(status_html))
    
    # Navigation buttons
    button_box = w.HBox([
        w.Button(description="🔑 Generate Keys", button_style="success", 
                 layout=w.Layout(width='180px', height='40px')),
        w.Button(description="🤝 Key Exchange", button_style="primary",
                 layout=w.Layout(width='180px', height='40px')),
        w.Button(description="✍️ Sign & Verify", button_style="warning",
                 layout=w.Layout(width='180px', height='40px')),
        w.Button(description="💬 Secure Messaging", button_style="info",
                 layout=w.Layout(width='180px', height='40px')),
        w.Button(description="🎬 Full Demo", button_style="danger",
                 layout=w.Layout(width='180px', height='40px'))
    ], layout=w.Layout(justify_content='center', margin='20px'))
    
    display(button_box)
    
    # Wire up button clicks
    button_box.children[0].on_click(lambda b: key_generation_screen())
    button_box.children[1].on_click(lambda b: key_exchange_screen())
    button_box.children[2].on_click(lambda b: signature_screen())
    button_box.children[3].on_click(lambda b: messaging_screen())
    button_box.children[4].on_click(lambda b: full_demo_screen())
    
    # Download session button if entities exist
    if current_atc and current_aircraft:
        download_section = w.VBox()
        
        download_btn = w.Button(
            description="📦 Download Session Archive",
            button_style="success",
            icon="download",
            layout=w.Layout(width="250px")
        )
        
        download_output = w.Output()
        
        def on_download(b):
            with download_output:
                clear_output()
                
                # Create archive
                archive_data = create_session_archive(
                    current_atc, 
                    current_aircraft,
                    demo_messages
                )
                
                # Display download link
                display(create_download_link(
                    archive_data,
                    f"aviation_crypto_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip",
                    "📦 Download Complete Session Archive"
                ))
        
        download_btn.on_click(on_download)
        
        download_section.children = [
            w.HTML("<hr style='margin: 30px 0;'>"),
            w.HTML("<h3 style='text-align: center;'>🗂️ Export Session Data</h3>"),
            w.HBox([download_btn], layout=w.Layout(justify_content='center')),
            download_output
        ]
        
        display(download_section)
    
    # Instructions
    instructions_html = """
    <div style='margin: 30px auto; max-width: 800px; padding: 20px; 
                background: #f8f9fa; border-radius: 10px;'>
        <h3>📋 How to Use This System:</h3>
        <ol style='line-height: 1.8;'>
            <li><strong>Generate Keys:</strong> Create hybrid cryptographic keys for both ATC and Aircraft</li>
            <li><strong>Key Exchange:</strong> Establish a quantum-secure session between ATC and Aircraft</li>
            <li><strong>Sign & Verify:</strong> Create and verify hybrid digital signatures on critical messages</li>
            <li><strong>Secure Messaging:</strong> Send encrypted communications using the established session</li>
            <li><strong>Full Demo:</strong> Watch a complete aviation communication scenario</li>
        </ol>
        <p style='margin-top: 20px;'><strong>💾 Downloads:</strong> All outputs can be downloaded to your computer for analysis or archival.</p>
    </div>
    """
    
    display(w.HTML(instructions_html))

# ============================================================================
# SECTION 5.2: KEY GENERATION SCREEN
# ============================================================================

def key_generation_screen():
    """
    Screen for generating hybrid cryptographic keys.
    """
    clear_output(wait=True)
    
    display(w.HTML("<h2>🔑 Hybrid Key Generation</h2>"))
    display(w.HTML("<p>Generate quantum-secure keys for ATC and Aircraft entities.</p>"))
    
    # Entity selection
    entity_type = w.Dropdown(
        options=["Air Traffic Control", "Aircraft"],
        value="Air Traffic Control",
        description="Entity Type:",
        style={'description_width': 'initial'}
    )
    
    entity_name = w.Text(
        value="JFK_Tower",
        placeholder="Enter entity identifier",
        description="Entity Name:",
        style={'description_width': 'initial'}
    )
    
    generate_btn = w.Button(
        description="Generate Keys",
        button_style="success",
        icon="key"
    )
    
    output_area = w.Output()
    
    display(w.VBox([
        entity_type,
        entity_name,
        generate_btn,
        output_area
    ]))
    
    back_button()
    
    def on_generate(b):
        global current_atc, current_aircraft
        
        with output_area:
            clear_output()
            
            # Determine entity type
            is_atc = entity_type.value == "Air Traffic Control"
            
            # Create entity
            entity = AviationEntity(
                entity_name.value,
                "ATC" if is_atc else "Aircraft"
            )
            
            # Generate keys
            timings = entity.generate_hybrid_keys()
            
            # Store entity
            if is_atc:
                current_atc = entity
            else:
                current_aircraft = entity
            
            # Display results
            display(status_box(f"Keys generated successfully for {entity_name.value}!", "success"))
            
            # Show timing metrics
            display(w.HTML("<h4>Performance Metrics:</h4>"))
            display_timing_metrics(timings)
            
            # Export keys
            public_keys = entity.export_public_keys()
            
            # Create download section
            display(w.HTML("<h4>📥 Download Options:</h4>"))
            
            # Create download container
            download_container = w.HBox()
            
            # Public keys download
            keys_json = json.dumps(public_keys, indent=2)
            download_container.children = [
                create_download_link(
                    keys_json.encode('utf-8'),
                    f"{entity_name.value}_public_keys.json",
                    "📄 Public Keys (JSON)"
                ),
                create_download_link(
                    keys_json.encode('utf-8'),
                    f"{entity_name.value}_public_keys.txt",
                    "📝 Public Keys (TXT)"
                )
            ]
            
            display(download_container)
            
            # Generate key report
            key_report = f"""
            <h1>Cryptographic Key Report</h1>
            <h2>Entity: {entity_name.value}</h2>
            <p><strong>Type:</strong> {entity.entity_type}</p>
            <p><strong>Generated:</strong> {public_keys['timestamp']}</p>
            
            <h3>Classical Keys</h3>
            <ul>
                <li><strong>ECDH Public Key:</strong> NIST P-384 Curve</li>
                <li><strong>ECDSA Public Key:</strong> NIST P-384 Curve</li>
            </ul>
            
            <h3>Post-Quantum Keys</h3>
            <ul>
                <li><strong>Kyber-768 Public Key:</strong> {len(decode_b64(public_keys['post_quantum']['kyber_public']))} bytes</li>
                <li><strong>Falcon-512 Public Key:</strong> {len(decode_b64(public_keys['post_quantum']['falcon_public']))} bytes</li>
            </ul>
            
            <h3>Security Level</h3>
            <ul>
                <li><strong>Classical Security:</strong> ~192-bit</li>
                <li><strong>Post-Quantum Security:</strong> NIST Level 3</li>
            </ul>
            """
            
            display(create_report_download(key_report, f"{entity_name.value}_key_report", "html"))
            
            display(w.HTML("<p style='color: #28a745; margin-top: 20px;'><strong>✓ Entity ready for secure communication!</strong></p>"))
    
    generate_btn.on_click(on_generate)

# ============================================================================
# SECTION 5.3: KEY EXCHANGE SCREEN
# ============================================================================

def key_exchange_screen():
    """
    Screen for performing hybrid key exchange.
    """
    clear_output(wait=True)
    
    display(w.HTML("<h2>🤝 Hybrid Key Exchange</h2>"))
    display(w.HTML("<p>Establish a quantum-secure session between ATC and Aircraft.</p>"))
    
    if not current_atc or not current_aircraft:
        display(status_box("Please generate keys for both ATC and Aircraft first!", "error"))
        back_button()
        return
    
    # Exchange options
    initiator = w.RadioButtons(
        options=['ATC initiates', 'Aircraft initiates'],
        value='ATC initiates',
        description='Exchange Mode:',
        style={'description_width': 'initial'}
    )
    
    exchange_btn = w.Button(
        description="Perform Key Exchange",
        button_style="primary",
        icon="sync"
    )
    
    output_area = w.Output()
    
    display(w.VBox([
        initiator,
        exchange_btn,
        output_area
    ]))
    
    back_button()
    
    def on_exchange(b):
        with output_area:
            clear_output()
            
            try:
                # Export public keys
                atc_public = current_atc.export_public_keys()
                aircraft_public = current_aircraft.export_public_keys()
                
                # Import peer keys
                current_atc.import_peer_public_keys(aircraft_public)
                current_aircraft.import_peer_public_keys(atc_public)
                
                # Determine who initiates
                atc_initiates = initiator.value == 'ATC initiates'
                
                display(w.HTML("<h3>🔄 Performing Hybrid Key Exchange...</h3>"))
                
                # Step 1: Initiator creates and sends Kyber ciphertext
                if atc_initiates:
                    display(w.HTML("<h4>Step 1: ATC initiating exchange</h4>"))
                    atc_key = current_atc.perform_hybrid_key_exchange(initiator=True)
                    
                    # Transfer Kyber ciphertext
                    current_aircraft.kyber_ciphertext = current_atc.kyber_ciphertext
                    
                    display(w.HTML("<h4>Step 2: Aircraft responding</h4>"))
                    aircraft_key = current_aircraft.perform_hybrid_key_exchange(initiator=False)
                else:
                    display(w.HTML("<h4>Step 1: Aircraft initiating exchange</h4>"))
                    aircraft_key = current_aircraft.perform_hybrid_key_exchange(initiator=True)
                    
                    # Transfer Kyber ciphertext
                    current_atc.kyber_ciphertext = current_aircraft.kyber_ciphertext
                    
                    display(w.HTML("<h4>Step 2: ATC responding</h4>"))
                    atc_key = current_atc.perform_hybrid_key_exchange(initiator=False)
                
                # Verify keys match
                if atc_key == aircraft_key:
                    display(status_box("✅ Secure session established successfully!", "success"))
                    display(w.HTML(f"<p><strong>Session key fingerprint:</strong> {encode_b64(atc_key)[:32]}...</p>"))
                    
                    # Show combined timings
                    all_timings = {}
                    all_timings.update({f"ATC {k}": v for k, v in current_atc.last_operation_timings.items()})
                    all_timings.update({f"Aircraft {k}": v for k, v in current_aircraft.last_operation_timings.items()})
                    
                    display(w.HTML("<h4>Performance Metrics:</h4>"))
                    display_timing_metrics(all_timings)
                    
                    # Generate key exchange report
                    kex_report = {
                        "timestamp": datetime.now().isoformat(),
                        "initiator": "ATC" if atc_initiates else "Aircraft",
                        "responder": "Aircraft" if atc_initiates else "ATC",
                        "session_key_fingerprint": encode_b64(atc_key)[:64],
                        "algorithms": {
                            "classical": "ECDH-P384",
                            "post_quantum": "Kyber-768",
                            "key_derivation": "HKDF-SHA256"
                        },
                        "timings": all_timings,
                        "status": "SUCCESS"
                    }
                    
                    # Download options
                    display(w.HTML("<h4>📥 Download Key Exchange Report:</h4>"))
                    display(create_report_download(
                        json.dumps(kex_report, indent=2),
                        "key_exchange_report",
                        "json"
                    ))
                else:
                    display(status_box("❌ Key exchange failed - keys don't match!", "error"))
                    
            except Exception as e:
                display(status_box(f"Error during key exchange: {str(e)}", "error"))
    
    exchange_btn.on_click(on_exchange)

# ============================================================================
# SECTION 5.4: SIGNATURE SCREEN - ENHANCED WITH DOWNLOADS
# ============================================================================

def signature_screen():
    """
    Screen for creating and verifying hybrid digital signatures with download capabilities.
    """
    clear_output(wait=True)
    
    display(w.HTML("<h2>✍️ Hybrid Digital Signatures</h2>"))
    display(w.HTML("<p>Sign critical aviation messages with quantum-secure signatures.</p>"))
    
    if not current_atc or not current_aircraft:
        display(status_box("Please generate keys first!", "error"))
        back_button()
        return
    
    # Create main container
    main_container = w.VBox()
    
    # === Section 1: Message Input ===
    message_section = w.VBox()
    
    signer_select = w.Dropdown(
        options=["ATC", "Aircraft"],
        value="ATC",
        description="Signer:"
    )
    
    message_input = w.Textarea(
        value="CLEARED FOR TAKEOFF RUNWAY 28L, WIND 270 AT 15",
        placeholder="Enter message to sign",
        description="Message:",
        layout=w.Layout(width='100%', height='80px')
    )
    
    message_section.children = [
        w.HTML("<h3>📝 Step 1: Select Signer and Message</h3>"),
        signer_select,
        message_input
    ]
    
    # === Section 2: Signature Creation ===
    signature_section = w.VBox()
    sign_btn = w.Button(
        description="Create Hybrid Signature",
        button_style="warning",
        icon="pencil",
        layout=w.Layout(width='200px')
    )
    
    sign_output = w.Output()
    
    signature_section.children = [
        w.HTML("<h3>🔐 Step 2: Hybrid Signature Creation</h3>"),
        sign_btn,
        sign_output
    ]
    
    # === Section 3: Signature Verification ===
    verification_section = w.VBox()
    verify_btn = w.Button(
        description="Verify Signature",
        button_style="info",
        icon="check-circle",
        layout=w.Layout(width='200px'),
        disabled=True  # Initially disabled
    )
    
    verify_output = w.Output()
    
    verification_section.children = [
        w.HTML("<h3>🔍 Step 3: Hybrid Signature Verification</h3>"),
        verify_btn,
        verify_output
    ]
    
    # Assemble main container
    main_container.children = [
        message_section,
        w.HTML("<hr style='margin: 20px 0;'>"),
        signature_section,
        w.HTML("<hr style='margin: 20px 0;'>"),
        verification_section
    ]
    
    display(main_container)
    back_button()
    
    # Storage for signed data
    signed_data_storage = {}
    
    def on_sign(b):
        """Real-time signature creation with step-by-step visualization"""
        with sign_output:
            clear_output()
            
            # Get signer
            signer = current_atc if signer_select.value == "ATC" else current_aircraft
            
            # Disable button during signing
            sign_btn.disabled = True
            sign_btn.description = "Signing..."
            
            # Step-by-step visualization
            display(w.HTML(f"<h4>🚀 Starting Hybrid Signature Process</h4>"))
            display(w.HTML(f"<p><strong>Signer:</strong> {signer.entity_name}</p>"))
            display(w.HTML(f"<p><strong>Message:</strong> '{message_input.value}'</p>"))
            
            time.sleep(0.5)  # Dramatic pause
            
            # Step 1: Hash Creation
            display(w.HTML("<h5>📊 Step 2.1: Creating Hybrid Hash</h5>"))
            progress_1 = w.IntProgress(value=0, min=0, max=100, description='Hashing:')
            display(progress_1)
            
            # Animate progress
            for i in range(0, 101, 20):
                progress_1.value = i
                time.sleep(0.1)
            
            display(w.HTML("""
            <div style='margin: 10px 0; padding: 10px; background: #e8f5e9; border-radius: 5px;'>
                <p>✓ SHA-256 hash computed</p>
                <p>✓ SHA-512 hash computed</p>
                <p>✓ Combined hash derived using HKDF</p>
            </div>
            """))
            
            time.sleep(0.5)
            
            # Step 2: Classical Signature
            display(w.HTML("<h5>🔏 Step 2.2: Creating Classical ECDSA Signature</h5>"))
            progress_2 = w.IntProgress(value=0, min=0, max=100, description='ECDSA:')
            display(progress_2)
            
            for i in range(0, 101, 25):
                progress_2.value = i
                time.sleep(0.1)
            
            display(w.HTML("""
            <div style='margin: 10px 0; padding: 10px; background: #e3f2fd; border-radius: 5px;'>
                <p>✓ ECDSA signature created using P-384 curve</p>
                <p>✓ Classical security established</p>
            </div>
            """))
            
            time.sleep(0.5)
            
            # Step 3: Post-Quantum Signature
            display(w.HTML("<h5>🛡️ Step 2.3: Creating Post-Quantum Falcon Signature</h5>"))
            progress_3 = w.IntProgress(value=0, min=0, max=100, description='Falcon:')
            display(progress_3)
            
            for i in range(0, 101, 20):
                progress_3.value = i
                time.sleep(0.1)
            
            display(w.HTML("""
            <div style='margin: 10px 0; padding: 10px; background: #f3e5f5; border-radius: 5px;'>
                <p>✓ Falcon-512 signature created</p>
                <p>✓ Quantum-resistant security established</p>
            </div>
            """))
            
            time.sleep(0.5)
            
            # Step 4: Signature Combination
            display(w.HTML("<h5>🔀 Step 2.4: Combining Signatures</h5>"))
            progress_4 = w.IntProgress(value=0, min=0, max=100, description='Combining:')
            display(progress_4)
            
            # Actually sign the message
            signed_data = signer.sign_message(message_input.value)
            
            for i in range(0, 101, 33):
                progress_4.value = i
                time.sleep(0.1)
            
            display(w.HTML("""
            <div style='margin: 10px 0; padding: 10px; background: #fff3e0; border-radius: 5px;'>
                <p>✓ Signatures XORed together</p>
                <p>✓ HKDF applied for final derivation</p>
                <p>✓ 512-bit hybrid signature created</p>
            </div>
            """))
            
            # Store for verification
            signed_data_storage['last_signed'] = signed_data
            
            # Display final result
            display(w.HTML(f"""
            <div style='margin: 20px 0; padding: 15px; background: #d4edda; border-radius: 5px; border: 1px solid #c3e6cb;'>
                <h4 style='color: #155724;'>✅ Hybrid Signature Created Successfully!</h4>
                <p><strong>Algorithm:</strong> {signed_data['algorithm']}</p>
                <p><strong>Signature Length:</strong> {len(decode_b64(signed_data['hybrid_signature']))} bytes</p>
                <p><strong>Components:</strong></p>
                <ul>
                    <li>ECDSA Component: {len(decode_b64(signed_data['components']['ecdsa']))} bytes</li>
                    <li>Falcon Component: {len(decode_b64(signed_data['components']['falcon']))} bytes</li>
                </ul>
                <p><strong>Time:</strong> {signed_data['timestamp']}</p>
            </div>
            """))
            
            # Show performance metrics
            display(w.HTML("<h5>⚡ Performance Metrics:</h5>"))
            display_timing_metrics(signer.last_operation_timings)
            
            # Download options
            display(w.HTML("<h4>📥 Download Signature Data:</h4>"))
            
            # Create signature package
            download_container = w.HBox()
            
            # JSON format
            download_container.children = [
                create_download_link(
                    json.dumps(signed_data, indent=2).encode('utf-8'),
                    f"hybrid_signature_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
                    "📄 Signature (JSON)"
                ),
                create_download_link(
                    signed_data['hybrid_signature'].encode('utf-8'),
                    f"signature_only_{datetime.now().strftime('%Y%m%d_%H%M%S')}.sig",
                    "🔏 Raw Signature"
                )
            ]
            
            display(download_container)
            
            # Re-enable button and enable verify button
            sign_btn.disabled = False
            sign_btn.description = "Create Hybrid Signature"
            verify_btn.disabled = False
            
            display(w.HTML("""
            <div style='margin: 10px 0; padding: 10px; background: #e1f5fe; border-radius: 5px;'>
                <p>💡 <strong>Next Step:</strong> Click "Verify Signature" to see the verification process</p>
            </div>
            """))
    
    def on_verify(b):
        """Real-time signature verification with step-by-step visualization"""
        if 'last_signed' not in signed_data_storage:
            with verify_output:
                clear_output()
                display(status_box("No signature to verify! Create a signature first.", "error"))
            return
        
        with verify_output:
            clear_output()
            
            # Get verifier (opposite of signer)
            signed_data = signed_data_storage['last_signed']
            verifier = current_aircraft if signer_select.value == "ATC" else current_atc
            
            # Disable button during verification
            verify_btn.disabled = True
            verify_btn.description = "Verifying..."
            
            display(w.HTML(f"<h4>🔍 Starting Hybrid Signature Verification</h4>"))
            display(w.HTML(f"<p><strong>Verifier:</strong> {verifier.entity_name}</p>"))
            display(w.HTML(f"<p><strong>Original Signer:</strong> {signed_data['signer']}</p>"))
            
            time.sleep(0.5)
            
            # Step 1: Hash Recreation
            display(w.HTML("<h5>📊 Step 3.1: Recreating Message Hash</h5>"))
            progress_1 = w.IntProgress(value=0, min=0, max=100, description='Hashing:')
            display(progress_1)
            
            for i in range(0, 101, 25):
                progress_1.value = i
                time.sleep(0.1)
            
            display(w.HTML("""
            <div style='margin: 10px 0; padding: 10px; background: #e8f5e9; border-radius: 5px;'>
                <p>✓ Message hashed with SHA-256</p>
                <p>✓ Message hashed with SHA-512</p>
                <p>✓ Hybrid hash recreated</p>
            </div>
            """))
            
            time.sleep(0.5)
            
            # Step 2: ECDSA Verification
            display(w.HTML("<h5>🔏 Step 3.2: Verifying ECDSA Component</h5>"))
            progress_2 = w.IntProgress(value=0, min=0, max=100, description='ECDSA:')
            display(progress_2)
            
            # Start actual verification
            is_valid, details = verifier.verify_signature(signed_data)
            
            for i in range(0, 101, 33):
                progress_2.value = i
                time.sleep(0.1)
            
            ecdsa_status = "✅ VALID" if details['ecdsa_valid'] else "❌ INVALID"
            display(w.HTML(f"""
            <div style='margin: 10px 0; padding: 10px; background: #e3f2fd; border-radius: 5px;'>
                <p>✓ Loaded signer's ECDSA public key</p>
                <p>✓ Verified signature against hybrid hash</p>
                <p><strong>ECDSA Result: {ecdsa_status}</strong></p>
            </div>
            """))
            
            time.sleep(0.5)
            
            # Step 3: Falcon Verification
            display(w.HTML("<h5>🛡️ Step 3.3: Verifying Falcon Component</h5>"))
            progress_3 = w.IntProgress(value=0, min=0, max=100, description='Falcon:')
            display(progress_3)
            
            for i in range(0, 101, 25):
                progress_3.value = i
                time.sleep(0.1)
            
            falcon_status = "✅ VALID" if details['falcon_valid'] else "❌ INVALID"
            display(w.HTML(f"""
            <div style='margin: 10px 0; padding: 10px; background: #f3e5f5; border-radius: 5px;'>
                <p>✓ Loaded signer's Falcon public key</p>
                <p>✓ Verified signature against hybrid hash</p>
                <p><strong>Falcon Result: {falcon_status}</strong></p>
            </div>
            """))
            
            time.sleep(0.5)
            
            # Step 4: Combined Verification
            display(w.HTML("<h5>🔀 Step 3.4: Validating Hybrid Construction</h5>"))
            progress_4 = w.IntProgress(value=0, min=0, max=100, description='Hybrid:')
            display(progress_4)
            
            for i in range(0, 101, 20):
                progress_4.value = i
                time.sleep(0.1)
            
            display(w.HTML("""
            <div style='margin: 10px 0; padding: 10px; background: #fff3e0; border-radius: 5px;'>
                <p>✓ Recreated signature combination (XOR)</p>
                <p>✓ Applied HKDF derivation</p>
                <p>✓ Compared with received signature</p>
            </div>
            """))
            
            # Display final result
            if is_valid:
                display(w.HTML(f"""
                <div style='margin: 20px 0; padding: 15px; background: #d4edda; border-radius: 5px; border: 2px solid #28a745;'>
                    <h4 style='color: #155724;'>✅ Signature Verification PASSED!</h4>
                    <p>The message is authentic and has not been tampered with.</p>
                    <ul>
                        <li>Hash Verification: ✅</li>
                        <li>ECDSA Component: ✅</li>
                        <li>Falcon Component: ✅</li>
                        <li>Hybrid Construction: ✅</li>
                    </ul>
                    <p><strong>Security Status:</strong> Protected against both classical and quantum attacks</p>
                </div>
                """))
            else:
                display(w.HTML(f"""
                <div style='margin: 20px 0; padding: 15px; background: #f8d7da; border-radius: 5px; border: 2px solid #dc3545;'>
                    <h4 style='color: #721c24;'>❌ Signature Verification FAILED!</h4>
                    <p>The signature is invalid or the message has been tampered with.</p>
                    <ul>
                        <li>Hash Verification: {'✅' if details['hash_valid'] else '❌'}</li>
                        <li>ECDSA Component: {'✅' if details['ecdsa_valid'] else '❌'}</li>
                        <li>Falcon Component: {'✅' if details['falcon_valid'] else '❌'}</li>
                    </ul>
                </div>
                """))
            
            # Show performance metrics
            display(w.HTML("<h5>⚡ Verification Performance:</h5>"))
            display_timing_metrics(details['timings'])
            
            # Generate verification report
            verification_report = {
                "timestamp": datetime.now().isoformat(),
                "verifier": verifier.entity_name,
                "signer": signed_data['signer'],
                "message": signed_data['message'],
                "verification_result": {
                    "overall": is_valid,
                    "hash_valid": details['hash_valid'],
                    "ecdsa_valid": details['ecdsa_valid'],
                    "falcon_valid": details['falcon_valid']
                },
                "algorithm": details['algorithm'],
                "performance": details['timings']
            }
            
            # Download verification report
            display(w.HTML("<h4>📥 Download Verification Report:</h4>"))
            display(create_report_download(
                json.dumps(verification_report, indent=2),
                f"verification_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
                "json"
            ))
            
            # Re-enable button
            verify_btn.disabled = False
            verify_btn.description = "Verify Signature"
            
            # Show explanation
            display(w.HTML(f"""
            <div style='margin: 20px 0; padding: 15px; background: #e8eaf6; border-radius: 5px;'>
                <h5>🔐 What Just Happened?</h5>
                <p>The hybrid signature verification process ensured:</p>
                <ol>
                    <li><strong>Message Integrity:</strong> The hash verification confirmed the message wasn't altered</li>
                    <li><strong>Classical Security:</strong> ECDSA verification provided current cryptographic protection</li>
                    <li><strong>Quantum Resistance:</strong> Falcon verification added post-quantum security</li>
                    <li><strong>Hybrid Assurance:</strong> Both signatures must be valid for overall verification to pass</li>
                </ol>
                <p>This dual-algorithm approach ensures security even if one algorithm is compromised by quantum computers.</p>
            </div>
            """))
    
    sign_btn.on_click(on_sign)
    verify_btn.on_click(on_verify)

# ============================================================================
# SECTION 5.5: MESSAGING SCREEN - ENHANCED WITH DOWNLOADS
# ============================================================================

def messaging_screen():
    """
    Screen for secure messaging using AES-GCM with download capabilities.
    """
    clear_output(wait=True)
    
    display(w.HTML("<h2>💬 Secure Messaging</h2>"))
    display(w.HTML("<p>Send encrypted messages using the established hybrid session key.</p>"))
    
    if not current_atc or not current_aircraft:
        display(status_box("Please setup entities first!", "error"))
        back_button()
        return
    
    if not current_atc.session_key or not current_aircraft.session_key:
        display(status_box("Please perform key exchange first!", "error"))
        back_button()
        return
    
    # Message interface
    sender_select = w.Dropdown(
        options=["ATC → Aircraft", "Aircraft → ATC"],
        value="ATC → Aircraft",
        description="Direction:"
    )
    
    message_input = w.Textarea(
        value="DESCEND TO FL250, MAINTAIN SPEED 280 KNOTS",
        placeholder="Enter message",
        description="Message:",
        layout=w.Layout(width='100%', height='80px')
    )
    
    send_btn = w.Button(
        description="Send Encrypted",
        button_style="primary",
        icon="send"
    )
    
    output_area = w.Output()
    
    display(w.VBox([
        sender_select,
        message_input,
        send_btn,
        output_area
    ]))
    
    back_button()
    
    def on_send(b):
        with output_area:
            clear_output()
            
            # Determine sender and receiver
            if "ATC →" in sender_select.value:
                sender = current_atc
                receiver = current_aircraft
            else:
                sender = current_aircraft
                receiver = current_atc
            
            display(w.HTML(f"<h3>📤 Sending from {sender.entity_name} to {receiver.entity_name}</h3>"))
            
            # Encrypt
            encrypted_data = sender.encrypt_message(message_input.value)
            
            display(status_box("Message encrypted", "success"))
            
            # Show encrypted data
            display(w.HTML(f"""
            <div style='margin: 10px 0; padding: 10px; background: #f8f9fa; 
                        border-radius: 5px; font-family: monospace;'>
                <h4>Encrypted Data:</h4>
                <p><strong>Ciphertext:</strong> {encrypted_data['ciphertext'][:64]}...</p>
                <p><strong>Nonce:</strong> {encrypted_data['nonce']}</p>
                <p><strong>Auth Tag:</strong> {encrypted_data['auth_tag']}</p>
            </div>
            """))
            
            # Download encrypted message
            display(w.HTML("<h4>📥 Download Encrypted Message:</h4>"))
            display(create_download_link(
                json.dumps(encrypted_data, indent=2).encode('utf-8'),
                f"encrypted_message_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
                "🔒 Encrypted Message"
            ))
            
            # Decrypt
            display(w.HTML(f"<h3>📥 Receiving at {receiver.entity_name}</h3>"))
            
            try:
                decrypted = receiver.decrypt_message(encrypted_data)
                
                display(status_box("Message decrypted successfully", "success"))
                display(w.HTML(f"""
                <div style='margin: 10px 0; padding: 15px; background: #e3f2fd; 
                            border-radius: 5px; border-left: 4px solid #2196f3;'>
                    <h4>Decrypted Message:</h4>
                    <p style='font-size: 16px; margin: 0;'>{decrypted}</p>
                </div>
                """))
                
                # Store for demo
                global demo_messages
                message_record = {
                    "sender": sender.entity_name,
                    "receiver": receiver.entity_name,
                    "message": decrypted,
                    "timestamp": encrypted_data['timestamp'],
                    "encrypted": True,
                    "verified": True
                }
                demo_messages.append(message_record)
                
                # Create communication transcript
                transcript = {
                    "communication_session": {
                        "timestamp": datetime.now().isoformat(),
                        "participants": {
                            "sender": sender.entity_name,
                            "receiver": receiver.entity_name
                        },
                        "encryption": {
                            "algorithm": "AES-256-GCM",
                            "key_exchange": "ECDH + Kyber-768"
                        },
                        "message": {
                            "plaintext": decrypted,
                            "ciphertext_sample": encrypted_data['ciphertext'][:64] + "...",
                            "nonce": encrypted_data['nonce'],
                            "auth_tag": encrypted_data['auth_tag'],
                            "timestamp": encrypted_data['timestamp']
                        },
                        "status": "SUCCESS"
                    }
                }
                
                # Download transcript
                display(w.HTML("<h4>📥 Download Communication Transcript:</h4>"))
                display(create_report_download(
                    json.dumps(transcript, indent=2),
                    f"communication_transcript_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
                    "json"
                ))
                
            except Exception as e:
                display(status_box(f"Decryption failed: {str(e)}", "error"))
    
    send_btn.on_click(on_send)

# ============================================================================
# SECTION 5.6: DEMO SCREEN - ENHANCED WITH COMPREHENSIVE REPORTING
# ============================================================================

def full_demo_screen():
    """
    Screen showing a complete aviation communication scenario with full reporting.
    """
    clear_output(wait=True)
    
    display(w.HTML("<h2>🎬 Full Aviation Security Demo</h2>"))
    display(w.HTML("<p>Watch a complete secure communication scenario between ATC and Aircraft.</p>"))
    
    # Demo control
    scenario_select = w.Dropdown(
        options=[
            "Emergency Landing Instructions",
            "Weather Advisory",
            "Route Change Authorization"
        ],
        value="Emergency Landing Instructions",
        description="Scenario:"
    )
    
    run_btn = w.Button(
        description="Run Demo",
        button_style="danger",
        icon="play"
    )
    
    output_area = w.Output()
    
    display(w.VBox([
        scenario_select,
        run_btn,
        output_area
    ]))
    
    back_button()
    
    def run_demo(b):
        with output_area:
            clear_output()
            
            # Demo data storage
            demo_data = {
                "scenario": scenario_select.value,
                "timestamp": datetime.now().isoformat(),
                "messages": [],
                "performance": {},
                "entities": {}
            }
            
            # Create fresh entities for demo
            display(w.HTML("<h3>🚁 Initializing Aviation Security System...</h3>"))
            time.sleep(0.5)
            
            # Step 1: Entity Creation
            display(w.HTML("<h4>Step 1: Creating Entities</h4>"))
            demo_atc = AviationEntity("MIAMI_CENTER", "ATC")
            demo_aircraft = AviationEntity("AA2521", "Aircraft")
            
            demo_data["entities"] = {
                "atc": {
                    "name": demo_atc.entity_name,
                    "type": demo_atc.entity_type
                },
                "aircraft": {
                    "name": demo_aircraft.entity_name,
                    "type": demo_aircraft.entity_type
                }
            }
            
            time.sleep(0.5)
            
            # Step 2: Key Generation
            display(w.HTML("<h4>Step 2: Generating Hybrid Keys</h4>"))
            
            start_time = time.time()
            atc_timings = demo_atc.generate_hybrid_keys()
            aircraft_timings = demo_aircraft.generate_hybrid_keys()
            demo_data["performance"]["Key Generation"] = time.time() - start_time
            
            time.sleep(0.5)
            
            # Step 3: Key Exchange
            display(w.HTML("<h4>Step 3: Establishing Secure Session</h4>"))
            
            start_time = time.time()
            
            # Exchange public keys
            atc_public = demo_atc.export_public_keys()
            aircraft_public = demo_aircraft.export_public_keys()
            
            demo_atc.import_peer_public_keys(aircraft_public)
            demo_aircraft.import_peer_public_keys(atc_public)
            
            # Perform key exchange
            demo_atc.perform_hybrid_key_exchange(initiator=True)
            demo_aircraft.kyber_ciphertext = demo_atc.kyber_ciphertext
            demo_aircraft.perform_hybrid_key_exchange(initiator=False)
            
            demo_data["performance"]["Key Exchange"] = time.time() - start_time
            
            display(status_box("✅ Quantum-secure session established", "success"))
            time.sleep(0.5)
            
            # Step 4: Scenario-specific communication
            display(w.HTML(f"<h4>Step 4: {scenario_select.value}</h4>"))
            
            if scenario_select.value == "Emergency Landing Instructions":
                messages = [
                    ("ATC", "AA2521 EMERGENCY: SQUAWK 7700, TURN LEFT HEADING 090"),
                    ("Aircraft", "SQUAWKING 7700, TURNING LEFT 090, AA2521"),
                    ("ATC", "AA2521 DESCEND AND MAINTAIN 3000, VECTORS FOR ILS 9R"),
                    ("Aircraft", "DESCENDING TO 3000, AA2521")
                ]
            elif scenario_select.value == "Weather Advisory":
                messages = [
                    ("ATC", "AA2521 WEATHER ADVISORY: SEVERE TURBULENCE AHEAD FL350-FL390"),
                    ("Aircraft", "COPY WEATHER, REQUESTING FL320, AA2521"),
                    ("ATC", "AA2521 DESCEND AND MAINTAIN FL320, REPORT SMOOTH"),
                    ("Aircraft", "DESCENDING FL320, WILCO, AA2521")
                ]
            else:  # Route Change
                messages = [
                    ("ATC", "AA2521 ROUTE AMENDMENT: PROCEED DIRECT WAYPOINT KAPPA"),
                    ("Aircraft", "DIRECT KAPPA, AA2521"),
                    ("ATC", "AA2521 AFTER KAPPA, J41 LIMA, THEN AS FILED"),
                    ("Aircraft", "KAPPA, J41 LIMA, THEN AS FILED, AA2521")
                ]
            
            # Process each message
            for sender_type, message in messages:
                time.sleep(1)  # Dramatic pause
                
                if sender_type == "ATC":
                    sender = demo_atc
                    receiver = demo_aircraft
                    icon = "🏢"
                else:
                    sender = demo_aircraft
                    receiver = demo_atc
                    icon = "✈️"
                
                display(w.HTML(f"<h5>{icon} {sender.entity_name}:</h5>"))
                
                # Performance tracking
                msg_start = time.time()
                
                # Sign message
                signed_data = sender.sign_message(message)
                
                # Encrypt message
                encrypted_data = sender.encrypt_message(message)
                
                # Receiver processes
                decrypted = receiver.decrypt_message(encrypted_data)
                is_valid, verify_details = receiver.verify_signature(signed_data)
                
                msg_time = time.time() - msg_start
                
                # Store message data
                demo_data["messages"].append({
                    "sender": sender.entity_name,
                    "receiver": receiver.entity_name,
                    "message": message,
                    "timestamp": encrypted_data['timestamp'],
                    "processing_time": msg_time,
                    "verified": is_valid,
                    "encryption": {
                        "algorithm": "AES-256-GCM",
                        "ciphertext_length": len(decode_b64(encrypted_data['ciphertext']))
                    },
                    "signature": {
                        "algorithm": signed_data['algorithm'],
                        "length": len(decode_b64(signed_data['hybrid_signature']))
                    }
                })
                
                display(w.HTML(f"""
                <div style='margin: 10px 0 20px 20px; padding: 10px; 
                            background: #f0f0f0; border-left: 3px solid #2196f3;'>
                    <p style='margin: 5px 0;'><strong>Message:</strong> {message}</p>
                    <p style='margin: 5px 0;'>🔐 Encrypted with AES-256-GCM</p>
                    <p style='margin: 5px 0;'>✍️ Signed with ECDSA + Falcon-512</p>
                    <p style='margin: 5px 0;'>✅ Verified: {is_valid}</p>
                    <p style='margin: 5px 0;'>⏱️ Processing time: {msg_time:.3f}s</p>
                </div>
                """))
            
            # Calculate total performance
            demo_data["performance"]["Total Messages"] = len(messages)
            demo_data["performance"]["Average Message Time"] = sum(m["processing_time"] for m in demo_data["messages"]) / len(messages)
            demo_data["status"] = "Completed Successfully"
            demo_data["atc_name"] = demo_atc.entity_name
            demo_data["aircraft_name"] = demo_aircraft.entity_name
            
            # Summary
            display(w.HTML("""
            <div style='margin: 30px 0; padding: 20px; background: #d4edda; 
                        border-radius: 10px; text-align: center;'>
                <h3 style='color: #155724;'>✅ Scenario Completed Successfully!</h3>
                <p>All communications were:</p>
                <ul style='text-align: left; display: inline-block;'>
                    <li>Encrypted with quantum-secure hybrid keys</li>
                    <li>Authenticated with hybrid digital signatures</li>
                    <li>Protected against current and future threats</li>
                </ul>
            </div>
            """))
            
            # Generate comprehensive report
            report_html = generate_demo_report(demo_data)
            
            # Download options
            display(w.HTML("<h3>📥 Download Demo Reports:</h3>"))
            
            download_container = w.HBox()
            
            # Multiple format options
            download_container.children = [
                create_report_download(report_html, f"demo_report_{scenario_select.value.replace(' ', '_')}", "html"),
                create_report_download(json.dumps(demo_data, indent=2), f"demo_data_{scenario_select.value.replace(' ', '_')}", "json"),
                create_report_download(
                    f"Aviation Security Demo Report\n" +
                    f"{'='*50}\n" +
                    f"Scenario: {scenario_select.value}\n" +
                    f"Date: {demo_data['timestamp']}\n" +
                    f"Status: {demo_data['status']}\n\n" +
                    f"Messages Exchanged: {len(demo_data['messages'])}\n" +
                    f"Average Processing Time: {demo_data['performance']['Average Message Time']:.3f}s\n",
                    f"demo_summary_{scenario_select.value.replace(' ', '_')}",
                    "txt"
                )
            ]
            
            display(download_container)
            
            # Create complete session archive
            display(w.HTML("<h4>📦 Complete Session Archive:</h4>"))
            
            archive_btn = w.Button(
                description="Generate Full Archive",
                button_style="success",
                icon="archive"
            )
            
            archive_output = w.Output()
            
            def on_archive(b):
                with archive_output:
                    clear_output()
                    
                    # Add demo data to message history
                    demo_atc.message_history.append({
                        "type": "demo_session",
                        "data": demo_data,
                        "timestamp": datetime.now().isoformat()
                    })
                    
                    # Create comprehensive archive
                    archive_data = create_session_archive(
                        demo_atc,
                        demo_aircraft,
                        demo_data["messages"]
                    )
                    
                    display(create_download_link(
                        archive_data,
                        f"aviation_demo_complete_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip",
                        "📦 Download Complete Archive (ZIP)"
                    ))
            
            archive_btn.on_click(on_archive)
            
            display(w.VBox([archive_btn, archive_output]))
    
    run_btn.on_click(run_demo)

# ============================================================================
# SECTION 6: GLOBAL STATE MANAGEMENT
# ============================================================================

# Global variables to store current entities
current_atc = None
current_aircraft = None
demo_messages = []  # Store messages for the scenario

# ============================================================================
# SECTION 7: MAIN EXECUTION
# ============================================================================

# Display welcome message
print("🚀 Aviation Hybrid Cryptography System Ready!")
print("=" * 50)
print("Enhanced with comprehensive download capabilities!")
print("All outputs can now be saved to your computer.")
print("=" * 50)

# Launch the home screen
home_screen()

HTML(value='<h2>🎬 Full Aviation Security Demo</h2>')

HTML(value='<p>Watch a complete secure communication scenario between ATC and Aircraft.</p>')

Button(button_style='info', description='← Back to Home', icon='home', layout=Layout(width='150px'), style=But…